In [1]:
import pandas as pd
import numpy as np

In [2]:
# ПОСЛЕДНИЙ МЕСЯЦ
# 16.06.2019
# from datetime import datetime
# from dateutil import relativedelta
# date1 = datetime.date.today()
# orders = pd.read_csv('orders.csv')
# orderLines = pd.read_csv('orderLines.csv')
# months = []
# for i in range(len(orders)):
#     date2 = datetime.strptime(str(orders.loc[i, 'DateTime']), '%d.%m.%Y')
#     months.append(relativedelta.relativedelta(date2, date1))
# orders['months'] = months
# orders = orders.loc[orders.months <= 1]
# ind = orders.index
# orderLines = orderLines.loc[ind]

### Искусственные данные ORDERS

In [3]:
# СТРОИМ ПРИМЕРНЫЕ ORDERS
orders = pd.DataFrame()
orders['DateTime'] = pd.Series(np.arange(100)+1)
orders['CustomerID'] = pd.Series(np.round(np.random.rand(100)*20))
orders['OrderID'] = pd.Series(np.round(np.random.rand(100)*10000))
orders.head()

,DateTime,CustomerID,OrderID
0,1,11.0,2394.0
1,2,7.0,149.0
2,3,18.0,284.0
3,4,17.0,3431.0
4,5,7.0,2493.0


### Искусственные данные ORDERLINES

In [5]:
# СТРОИМ ПРИМЕРНЫЕ ORDERLINES
orderLines = pd.DataFrame()
orderLines['OrderID'] = orders['OrderID']
orderLines['ProductID'] = pd.Series(np.round(np.random.rand(100)*20))
orderLines['Price'] = orderLines['ProductID']*10
orderLines.head()

,OrderID,ProductID,Price
0,2394.0,16.0,160.0
1,149.0,15.0,150.0
2,284.0,17.0,170.0
3,3431.0,6.0,60.0
4,2493.0,19.0,190.0


### Отчет

In [10]:
# СТРОИМ ОТЧЕТ
report = pd.DataFrame()

report['most_popular_products_id'] = orderLines['ProductID'].value_counts().keys()
report['total_value'] = orderLines['ProductID'].value_counts().values

for i in range(len(report)):
    report.total_value.loc[i] *= orderLines.loc[orderLines.ProductID == 
                                                report.most_popular_products_id.loc[i], 'Price'].values[0]
total = []

for i in report.most_popular_products_id:
    total_p = 0
    count = 0
    
    for j in orderLines.index:
        if orderLines.loc[j, 'ProductID'] == i:
            order_id = orderLines.loc[j, 'OrderID']
            total_p += sum(orderLines.loc[orderLines.OrderID == order_id, 'Price'])
            count += 1
    total.append(total_p/count)

report['mean_value_for_order'] = total

report.head()

,most_popular_products_id,total_value,mean_value_for_order
0,9.0,810.0,90.0
1,12.0,1080.0,120.0
2,10.0,800.0,100.0
3,15.0,1050.0,150.0
4,18.0,1260.0,180.0
